
# STRUCTURE

In [46]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [47]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [48]:
# the path to your HDF5 formatted snps file
data = "./M_ru_3rm_v9.snps.hdf5"

In [72]:
imap = {
#"ref": ["reference"],
#"Inam": ['M_ru_A7875_In','M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'],
#"Puru": ['M_ru_A10311_pu','M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'],
#"JiGu": ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu'],
"Mach": ['M_ru_T368_ma', 'M_ru_T381_ma', 'M_ru_T476_ma', 'M_ru_T494_ma'],#'M_ru_T13253_ma','M_ru_A474_ma', 'M_ru_T3164_ma', 'M_ru_J265_ma', 
"Roar": ['M_ru_J640_roar','M_ru_J676_roar'],
"ArSu": ['M_ru_80819_arsu', 'M_ru_81347_arsu', 'M_ru_85426_arsu'],
"SuTa": [ 'M_ru_85919_suta','M_ru_77750_suta', 'M_ru_78182_suta', 'M_ru_A11834_suta', 'M_ru_A15176_suta', 'M_ru_A5487_suta', 'M_ru_T10184_suta', 'M_ru_T11780_suta', 'M_ru_T14532_suta', 'M_ru_T14622_suta', 'M_ru_T753_suta', 'M_ru_A9903_pa'],
"Para": ['M_ru_A16195_pa','M_ru_A9235_pa', 'M_ru_T11079_pa', 'M_ru_T11238_pa', 'M_ru_T12541_pa', 'M_ru_T1649_pa', 'M_ru_T16553_pa', 'M_ru_T19782_pa', 'M_ru_T6500_pa', 'M_ru_T6577_pa']
}
 
# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [5]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="M_ru_east_str",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

Samples: 31
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 9343
Filtered (mincov): 726024
Filtered (minmap): 770178
Filtered (combined): 771026
Sites after filtering: 48908
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [6]:
struct.mainparams.burnin = 25000
struct.mainparams.numreps = 100000
struct.write_structure_files(abs)

('/array1/lmusher/rio_roosevelt_outfiles/M_ru_3rm_v9_outfiles/analysis-structure/tmp-M_ru_east_str-<built-in function abs>-1.mainparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/M_ru_3rm_v9_outfiles/analysis-structure/tmp-M_ru_east_str-<built-in function abs>-1.extraparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/M_ru_3rm_v9_outfiles/analysis-structure/tmp-M_ru_east_str-<built-in function abs>-1.strfile.txt')

In [ ]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8], ipyclient=ipyclient, force=True)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[######              ]  34% 2:02:26 | running 70 structure jobs 

In [50]:
struct = ipa.structure(
    data=data, 
    name="M_ru_east_str", 
    workdir="analysis-structure",
    imap=imap,
    load_only=True,
)

70 previous results loaded for run [M_ru_east_str]


In [51]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8])
etable

,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,10,0.000,-14427.48,343.098,0.00,0.00
3,10,3.095,-15400.72,317.891,-973.24,983.89
4,10,8.775,-15390.07,297.860,10.65,2613.80
5,10,0.108,-17993.22,8583.985,-2603.15,925.26
6,10,0.366,-19671.11,6135.162,-1677.89,2246.33
7,10,0.017,-23595.33,12732.423,-3924.22,217.88
8,10,0.000,-27737.43,10549.358,-4142.10,0.00


In [52]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc3554c80884a448a8c71509014fd10ba" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 K (N ancestral populations) 15000 20000 25000 30000 estLnProbMean 0 4 8 12 deltaK

In [53]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [54]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [55]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t937f634341d246468144fe1444bb2fe4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_J640_roar M_ru_J676_roar M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_77750_suta M_ru_78182_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A5487_suta M_ru_T10184_suta M_ru_T11780_suta M_ru_T14532_suta M_ru_T14622_suta M_ru_T753_suta M_ru_A9903_pa M_ru_A16195_pa M_ru_A9235_pa M_ru_T11079_pa M_ru_T11238_pa M_ru_T12541_pa M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T6500_pa M_ru_T6577_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma 0.0 0.5 1.0

In [56]:
k = 3
table = struct.get_clumpp_table(k)

[K3] 10/10 results permuted across replicates (max_var=0).


In [57]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [58]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t985bcda8dc3b48359b3a205470c80134" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_J640_roar M_ru_J676_roar M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_77750_suta M_ru_78182_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A5487_suta M_ru_T10184_suta M_ru_T11780_suta M_ru_T14532_suta M_ru_T14622_suta M_ru_T753_suta M_ru_A9903_pa M_ru_A16195_pa M_ru_A9235_pa M_ru_T11079_pa M_ru_T11238_pa M_ru_T12541_pa M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T6500_pa M_ru_T6577_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma 0.0 0.5 1.0

In [73]:
k = 4
table = struct.get_clumpp_table(k)

[K4] 10/10 results permuted across replicates (max_var=0).


In [74]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [71]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t78a27546924a46e193fed9a3149f6817" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_J640_roar M_ru_J676_roar M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_77750_suta M_ru_78182_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A5487_suta M_ru_T10184_suta M_ru_T11780_suta M_ru_T14532_suta M_ru_T14622_suta M_ru_T753_suta M_ru_A9903_pa M_ru_A16195_pa M_ru_A9235_pa M_ru_T11079_pa M_ru_T11238_pa M_ru_T12541_pa M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T6500_pa M_ru_T6577_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma 0.0 0.5 1.0

In [75]:
k = 5
table = struct.get_clumpp_table(k)

[K5] 10/10 results permuted across replicates (max_var=0).


In [76]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [77]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tdb2d7f8e679a470bba40b91daa9fcdea" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_J640_roar M_ru_J676_roar M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_77750_suta M_ru_78182_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A5487_suta M_ru_T10184_suta M_ru_T11780_suta M_ru_T14532_suta M_ru_T14622_suta M_ru_T753_suta M_ru_A9903_pa M_ru_A16195_pa M_ru_A9235_pa M_ru_T11079_pa M_ru_T11238_pa M_ru_T12541_pa M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T6500_pa M_ru_T6577_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma 0.0 0.5 1.0

In [78]:
k = 6
table = struct.get_clumpp_table(k)

[K6] 10/10 results permuted across replicates (max_var=0).


In [79]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [80]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tbfa38c189ae64f0cb59b0d7c25698332" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_J640_roar M_ru_J676_roar M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_77750_suta M_ru_78182_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A5487_suta M_ru_T10184_suta M_ru_T11780_suta M_ru_T14532_suta M_ru_T14622_suta M_ru_T753_suta M_ru_A9903_pa M_ru_A16195_pa M_ru_A9235_pa M_ru_T11079_pa M_ru_T11238_pa M_ru_T12541_pa M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T6500_pa M_ru_T6577_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma 0.0 0.5 1.0

# PCA

In [81]:
#!conda install scikit-learn -c conda-forge -y

In [82]:
import ipyrad.analysis as ipa
import pandas as pd
import toyplot

In [83]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
)

Samples: 31
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 9343
Filtered (mincov): 500244
Filtered (minmap): 770178
Filtered (combined): 771026
Sites after filtering: 48908
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [84]:
# run the PCA analysis
pca.run()

Subsampling SNPs: 6264/48908


In [85]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2b31d85c7110>,
 <toyplot.mark.Point at 0x2b31d8dcf050>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t85ab546e55f64f8cbfb6a0ec8407f20a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -5 0 5 10 15 PC0 (46.0%) explained -4 0 4 8 PC1 (4.1%) explained Roar ArSu SuTa Para Mach

In [86]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("M_ru_pca_analysis.csv")

# show the first ten samples and the first 10 PC axes
df.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
M_ru_77750_suta,-6.37,-1.73,0.68,2.87,1.76,-0.95,-0.01,-1.15,-0.11,-1.61
M_ru_78182_suta,-7.20,-1.71,-0.69,1.97,5.16,-0.97,-1.74,-0.03,0.07,5.36
M_ru_80819_arsu,-6.82,-1.61,-0.64,1.12,0.81,3.33,3.64,5.16,-1.30,-1.04
M_ru_81347_arsu,-6.73,-1.11,-0.16,2.80,-0.76,1.40,2.51,3.62,-1.19,2.15
M_ru_85426_arsu,-6.18,-2.16,1.90,3.69,-2.25,6.30,-2.51,-1.79,-3.40,-0.96
M_ru_85919_suta,-7.01,-2.25,0.52,-0.58,6.10,-0.61,-0.82,0.29,0.55,-1.67
M_ru_A11834_suta,-6.77,-2.60,-2.90,-0.12,1.43,0.25,-2.50,-5.11,-1.27,-1.89
M_ru_A15176_suta,-5.97,-3.33,0.67,-4.56,-0.13,1.01,-1.40,0.70,3.48,-2.80
M_ru_A16195_pa,13.94,0.28,-0.15,0.01,0.03,-0.16,0.25,-1.04,-0.23,0.19
M_ru_A5487_suta,-6.83,-1.78,-1.89,-1.28,-4.29,-3.63,2.48,-3.23,-2.48,1.93


# Subsampling with replication

In [87]:
# plot PC axes 0 and 2 with many replicate subsamples
pca.run(nreplicates=25, seed=12345)
pca.draw(0, 1);

Subsampling SNPs: 6264/48908


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t5190d626f9ea4f23b2449cc05c3b73ad" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -5 0 5 10 15 PC0 (42.4%) explained -5 0 5 10 PC1 (5.3%) explained Roar ArSu SuTa Para Mach

# Kmeans imputation (integer)

In [88]:
# kmeans imputation 
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)

# run and draw results for kmeans clustering into 7 groups
pca3.run(nreplicates=25, seed=123)
pca3.draw(0, 1);

Subsampling SNPs: 6264/48908


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t7a65d46be99342fc80b6089793f03667" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -5 0 5 10 15 PC0 (43.7%) explained -5 0 5 10 PC1 (5.0%) explained Roar ArSu SuTa Para Mach

In [89]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca3.draw(0, 2)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "M_ru_PCA-kmeans-7.pdf")

# T-SNE (ASSESSING COMPONENT LOADINGS)

In [90]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [91]:
pca.run_tsne(subsample=True, perplexity=3.0, n_iter=100000, seed=623)

Subsampling SNPs: 6264/48908


In [92]:
pca.draw(size=8);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t380f8036827643ab9688a148de5961ec" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_77750_suta M_ru_78182_suta M_ru_80819_arsu M_ru_81347_arsu M_ru_85426_arsu M_ru_85919_suta M_ru_A11834_suta M_ru_A15176_suta M_ru_A16195_pa M_ru_A5487_suta M_ru_A9235_pa M_ru_A9903_pa M_ru_J640_roar M_ru_J676_roar M_ru_T10184_suta M_ru_T11079_pa M_ru_T11238_pa M_ru_T11780_suta M_ru_T12541_pa M_ru_T14532_suta M_ru_T14622_suta M_ru_T1649_pa M_ru_T16553_pa M_ru_T19782_pa M_ru_T368_ma M_ru_T381_ma M_ru_T476_ma M_ru_T494_ma M_ru_T6500_pa M_ru_T6577_pa M_ru_T753_suta -200 -100 0 100 200 300 TNSE component 1 -400 -200 0 200 400 TNSE component 2 Roar ArSu SuTa Para Mach

In [93]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca.draw(size=8)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "M_ru_TSNE_perp3.pdf")

# GENETIC DISTNANCES

In [94]:
import ipyrad.analysis as ipa
import toyplot

In [95]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
)
dist.run()

Samples: 31
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 9343
Filtered (mincov): 500244
Filtered (minmap): 770178
Filtered (combined): 771026
Sites after filtering: 48908
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


In [96]:
# save to a CSV file
dist.dists.to_csv("M_ru_distances.csv")

In [97]:
# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "M_ru_distances_eems.csv",
    header=None,
    index=False,
    sep=" ",
)

In [101]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [102]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="tfcd9a962b5b34e7384b3fa819dac2693" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.038337 0.040811 0.042222 0.041609 0.042733 0.041302 0.042611 0.042324 0.041977 0.042386 0.042467 0.042345 0.042059 0.042754 0.040811 0.042427 0.098655 0.099575 0.100720 0.102335 0.099309 0.100004 0.100025 0.101517 0.101354 0.101538 0.040239 0.042018 0.040730 0.040566 M_ru_J640_roar 1 0.038337 0.000000 0.043776 0.044573 0.043388 0.044921 0.043940 0.045616 0.045657 0.043265 0.044451 0.043919 0.043020 0.044492 0.044942 0.044144 0.044860 0.101619 0.102008 0.102621 0.104523 0.101333 0.102478 0.102662 0.103705 0.103828 0.104012 0.043326 0.044410 0.044144 0.043408 M_ru_J676_roar 2 0.040811 0.043776 0.000000 0.037356 0.035843 0.037540 0.037417 0.038235 0.037826 0.036661 0.037683 0.036538 0.037315 0.037029 0.036456 0.037008 0.035843 0.096692 0.097121 0.098143 0.099841 0.096201 0.097060 0.097326 0.098900 0.098777 0.099125 0.045023 0.045903 0.045759 0.045473 M_ru_80819_arsu 3 0.042222 0.044573 0.037356 0.000000 0.038440 0.038664 0.040627 0.039769 0.040218 0.038399 0.038726 0.038440 0.038399 0.039053 0.038399 0.038501 0.038031 0.098143 0.098818 0.099350 0.101251 0.098225 0.099329 0.099186 0.100393 0.100597 0.100945 0.046066 0.047272 0.046475 0.046434 M_ru_81347_arsu 4 0.041609 0.043388 0.035843 0.038440 0.000000 0.038869 0.038746 0.038828 0.038705 0.037090 0.038317 0.038112 0.037049 0.037867 0.037581 0.037192 0.036599 0.096467 0.097019 0.097959 0.099779 0.096671 0.097285 0.097387 0.099207 0.099370 0.099350 0.046516 0.046455 0.047456 0.045534 M_ru_85426_arsu 5 0.042733 0.044921 0.037540 0.038664 0.038869 0.000000 0.039625 0.039993 0.039585 0.038624 0.039278 0.038910 0.038296 0.038828 0.039360 0.038317 0.038419 0.098287 0.098961 0.099411 0.101149 0.098327 0.099104 0.099411 0.100413 0.100168 0.100924 0.046373 0.046925 0.047272 0.046455 M_ru_85919_suta 6 0.041302 0.043940 0.037417 0.040627 0.038746 0.039625 0.000000 0.039421 0.039135 0.040177 0.039360 0.039441 0.039114 0.040300 0.039196 0.038358 0.039891 0.098777 0.099820 0.099943 0.102253 0.098982 0.099186 0.099738 0.101558 0.101354 0.101742 0.046823 0.048233 0.048377 0.047559 M_ru_77750_suta 7 0.042611 0.045616 0.038235 0.039769 0.038828 0.039993 0.039421 0.000000 0.040239 0.040505 0.040300 0.039032 0.039523 0.039605 0.040668 0.038480 0.038787 0.098818 0.099656 0.100025 0.101967 0.098941 0.099841 0.099738 0.101108 0.100986 0.101415 0.047763 0.048111 0.047968 0.047640 M_ru_78182_suta 8 0.042324 0.045657 0.037826 0.040218 0.038705 0.039585 0.039135 0.040239 0.000000 0.038092 0.037765 0.037683 0.038133 0.038378 0.038828 0.036967 0.037765 0.098777 0.099656 0.099984 0.101681 0.099145 0.099595 0.099738 0.100822 0.101272 0.101538 0.046945 0.048438 0.047640 0.047313 M_ru_A11834_suta 9 0.041977 0.043265 0.036661 0.038399 0.037090 0.038624 0.040177 0.040505 0.038092 0.000000 0.037376 0.036599 0.036477 0.037172 0.036354 0.035352 0.035454 0.095853 0.096528 0.096896 0.098839 0.095853 0.096303 0.096610 0.098348 0.097775 0.098368 0.045412 0.045964 0.046189 0.045698 M_ru_A15176_suta 10 0.042386 0.044451 0.037683 0.038726 0.038317 0.039278 0.039360 0.040300 0.037765 0.037376 0.000000 0.036722 0.037622 0.037295 0.037867 0.036538 0.036967 0.097694 0.098532 0.099227 0.100965 0.097939 0.098593 0.099104 0.100433 0.100311 0.100495 0.045494 0.046904 0.047743 0.046966 M_ru_A5487_suta 11 0.042467 0.043919 0.036538 0.038440 0.038112 0.038910 0.039441 0.039032 0.037683 0.036599 0.036722 0.000000 0.036436 0.037131 0.037131 0.035925 0.035700 0.097530 0.097919 0.098368 0.100597 0